In [1]:
import math

import numpy as np
import pandas as pd
import scipy as sp
from numpy.random import random
import tensorflow as tf

c:\users\songli3\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
SPLIT = 2

In [3]:
class SVD:
    def __init__(self, _user_num, _item_num,_batch_size=128, _learning_rate=0.0005, _lambda=0.001, _hidden_dimension=64):
        self._learning_rate = _learning_rate
        self._lambda = _lambda
        self._hidden_dimension = _hidden_dimension
        self._batch_size = _batch_size
        self._u = 2.5
        self._user_num = _user_num
        self._item_num = _item_num
        self._b_i = tf.get_variable("b_i", shape=(_item_num))
        self._b_u = tf.get_variable("b_u", shape=(_user_num))
        self._q = tf.get_variable("q", shape=(_item_num, _hidden_dimension), dtype=tf.float32,\
                                  initializer=tf.orthogonal_initializer(gain=1.0, seed=None, dtype=tf.float32))
        self._p = tf.get_variable("p", shape=(_user_num, _hidden_dimension), dtype=tf.float32,\
                                  initializer=tf.orthogonal_initializer(gain=1.0, seed=None, dtype=tf.float32))
        self.build_model()
        return
    
    def build_model(self):
        self._input_u = tf.placeholder(dtype=tf.int32, shape=[None, 1])
        self._input_i = tf.placeholder(dtype=tf.int32, shape=[None, 1])
        self._input_r = tf.placeholder(dtype=tf.float32, shape=[None, 1])
        
        input_u_v = tf.nn.embedding_lookup(self._p, self._input_u)
        input_i_v = tf.nn.embedding_lookup(self._q, self._input_i)
        input_u_v = tf.reshape(input_u_v, shape=(-1, self._hidden_dimension))
        input_i_v = tf.reshape(input_i_v, shape=(-1, self._hidden_dimension))
        
        ratings = tf.multiply(input_u_v, input_i_v)
        
        ratings = tf.reduce_sum(ratings, axis=1, keepdims=True) 
        
        b_i = tf.nn.embedding_lookup(self._b_i, self._input_i)
        b_u = tf.nn.embedding_lookup(self._b_u, self._input_u)
        
        remains = self._input_r - self._u - b_i - b_u - ratings
        self._loss = tf.reduce_sum( tf.pow(remains, 2), axis=0)
        tf.summary.histogram("loss", self._loss)
        reg = tf.reduce_sum(tf.pow(b_i, 2), axis=0) + \
            tf.reduce_sum (tf.pow(b_u, 2), axis=0) + \
            tf.reduce_sum(tf.pow(input_u_v, 2)) + tf.reduce_sum(tf.pow(input_i_v,2))
        self._total_loss = self._loss + reg * self._lambda
        
        self._optimizer = tf.train.GradientDescentOptimizer(learning_rate=self._learning_rate)
        self._train_op = self._optimizer.minimize(self._total_loss)
    
    def train(self):
        log_dir = "."
        self._sess = tf.Session()
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(log_dir + '/train', self._sess.graph)
        test_writer = tf.summary.FileWriter(log_dir + '/test')
        
        self._sess.run(tf.global_variables_initializer())
        step = 0
        for T in range(20):
            for u, i, r in next_batch(self._batch_size):
                # print("u", u)
                # print("i", i)
                
                rs, _, _, _loss = self._sess.run([merged, self._train_op, self._total_loss, self._loss],
                                         feed_dict={self._input_u:np.reshape(np.array(u),(self._batch_size,1)), 
                                                    self._input_i:np.reshape(np.array(i),(self._batch_size, 1)), 
                                                    self._input_r:np.reshape(np.array(r),(self._batch_size, 1))})
                # self._sess.run(merged)
                train_writer.add_summary(rs, step)
                print("[step %d]: loss %f" % (step, math.sqrt(_loss / self._batch_size) ))
                step += 1
    
    def test(self):
        df = pd.read_csv("../data/ml-100k/u%d.test" % SPLIT,sep='\t',header=None)
        data = df.values
        n = len(df)
        u = data[:, 0].tolist()
        i = data[:, 1].tolist()
        r = data[:, 2].tolist()
        _loss = self._sess.run(self._loss, 
                              feed_dict={self._input_u: np.reshape(np.array(u), (n, 1)),
                                        self._input_i: np.reshape(np.array(i), (n, 1)),
                                        self._input_r: np.reshape(np.array(r), (n, 1))})
        print("test loss: ", math.sqrt(_loss/n))

In [4]:
def next_batch(batch_size):
    start = 0
    df = pd.read_csv("../data/ml-100k/u%d.base" % SPLIT, sep='\t', header=None)
    total_items = len(df)
    df.iloc[:, 0] = df.iloc[:, 0] - 1
    df.iloc[:, 1] = df.iloc[:, 1] - 1
    raw_data = df.values.tolist()
    np.random.shuffle(raw_data)
    df = pd.DataFrame(raw_data)
    while (start + batch_size <= total_items ):
        data = df.iloc[start: start+batch_size,].values
        u = data[:,0].tolist()
        i = data[:, 1].tolist()
        r = data[:, 2].tolist()
        start += batch_size
        yield u, i, r

In [5]:
svd = SVD(943, 1682)

In [6]:
svd.train()

[step 0]: loss 1.520878
[step 1]: loss 1.564926
[step 2]: loss 1.604235
[step 3]: loss 1.580021
[step 4]: loss 1.421931
[step 5]: loss 1.598380
[step 6]: loss 1.455625
[step 7]: loss 1.454802
[step 8]: loss 1.394791
[step 9]: loss 1.554018
[step 10]: loss 1.531717
[step 11]: loss 1.574490
[step 12]: loss 1.498589
[step 13]: loss 1.406355
[step 14]: loss 1.463355
[step 15]: loss 1.467397
[step 16]: loss 1.564417
[step 17]: loss 1.459251
[step 18]: loss 1.488469
[step 19]: loss 1.463048
[step 20]: loss 1.468712
[step 21]: loss 1.489636
[step 22]: loss 1.516590
[step 23]: loss 1.559193
[step 24]: loss 1.515298
[step 25]: loss 1.536511
[step 26]: loss 1.585557
[step 27]: loss 1.469550
[step 28]: loss 1.567619
[step 29]: loss 1.487696
[step 30]: loss 1.518940
[step 31]: loss 1.452375
[step 32]: loss 1.514216
[step 33]: loss 1.476184
[step 34]: loss 1.545978
[step 35]: loss 1.551792
[step 36]: loss 1.577677
[step 37]: loss 1.452326
[step 38]: loss 1.602340
[step 39]: loss 1.529966
[step 40]:

[step 335]: loss 1.342809
[step 336]: loss 1.370579
[step 337]: loss 1.401354
[step 338]: loss 1.416468
[step 339]: loss 1.473420
[step 340]: loss 1.323327
[step 341]: loss 1.266482
[step 342]: loss 1.334959
[step 343]: loss 1.494892
[step 344]: loss 1.365507
[step 345]: loss 1.322593
[step 346]: loss 1.483956
[step 347]: loss 1.373839
[step 348]: loss 1.271493
[step 349]: loss 1.407730
[step 350]: loss 1.416554
[step 351]: loss 1.282492
[step 352]: loss 1.423748
[step 353]: loss 1.334348
[step 354]: loss 1.366271
[step 355]: loss 1.438297
[step 356]: loss 1.434524
[step 357]: loss 1.407229
[step 358]: loss 1.533242
[step 359]: loss 1.411035
[step 360]: loss 1.440281
[step 361]: loss 1.450239
[step 362]: loss 1.400123
[step 363]: loss 1.362773
[step 364]: loss 1.426596
[step 365]: loss 1.364589
[step 366]: loss 1.376507
[step 367]: loss 1.454143
[step 368]: loss 1.355330
[step 369]: loss 1.404205
[step 370]: loss 1.243831
[step 371]: loss 1.285809
[step 372]: loss 1.467259
[step 373]: 

[step 782]: loss 1.232043
[step 783]: loss 1.277936
[step 784]: loss 1.244234
[step 785]: loss 1.229309
[step 786]: loss 1.270081
[step 787]: loss 1.279872
[step 788]: loss 1.228006
[step 789]: loss 1.297526
[step 790]: loss 1.236463
[step 791]: loss 1.283164
[step 792]: loss 1.226108
[step 793]: loss 1.352228
[step 794]: loss 1.243356
[step 795]: loss 1.308237
[step 796]: loss 1.218232
[step 797]: loss 1.245795
[step 798]: loss 1.268196
[step 799]: loss 1.292077
[step 800]: loss 1.302378
[step 801]: loss 1.157604
[step 802]: loss 1.270893
[step 803]: loss 1.256925
[step 804]: loss 1.217226
[step 805]: loss 1.200694
[step 806]: loss 1.333066
[step 807]: loss 1.139465
[step 808]: loss 1.378846
[step 809]: loss 1.331451
[step 810]: loss 1.263536
[step 811]: loss 1.207572
[step 812]: loss 1.278332
[step 813]: loss 1.298950
[step 814]: loss 1.283193
[step 815]: loss 1.221423
[step 816]: loss 1.276840
[step 817]: loss 1.327883
[step 818]: loss 1.226935
[step 819]: loss 1.141874
[step 820]: 

[step 1240]: loss 1.252693
[step 1241]: loss 1.249340
[step 1242]: loss 1.278394
[step 1243]: loss 1.161787
[step 1244]: loss 1.232272
[step 1245]: loss 1.127485
[step 1246]: loss 1.278667
[step 1247]: loss 1.102986
[step 1248]: loss 1.159451
[step 1249]: loss 1.152678
[step 1250]: loss 1.162471
[step 1251]: loss 1.195386
[step 1252]: loss 1.117284
[step 1253]: loss 1.261664
[step 1254]: loss 1.156611
[step 1255]: loss 1.181346
[step 1256]: loss 1.217878
[step 1257]: loss 1.129214
[step 1258]: loss 1.245327
[step 1259]: loss 1.179026
[step 1260]: loss 1.205248
[step 1261]: loss 1.136987
[step 1262]: loss 1.156231
[step 1263]: loss 1.164144
[step 1264]: loss 1.243632
[step 1265]: loss 1.184595
[step 1266]: loss 1.271325
[step 1267]: loss 1.291767
[step 1268]: loss 1.156683
[step 1269]: loss 1.202753
[step 1270]: loss 1.239814
[step 1271]: loss 1.223121
[step 1272]: loss 1.112613
[step 1273]: loss 1.122318
[step 1274]: loss 1.223893
[step 1275]: loss 1.140039
[step 1276]: loss 1.124996
[

[step 1597]: loss 1.170648
[step 1598]: loss 1.118373
[step 1599]: loss 1.174325
[step 1600]: loss 1.216211
[step 1601]: loss 1.216980
[step 1602]: loss 1.166886
[step 1603]: loss 1.215021
[step 1604]: loss 1.068094
[step 1605]: loss 1.200884
[step 1606]: loss 1.219330
[step 1607]: loss 1.227286
[step 1608]: loss 1.175493
[step 1609]: loss 1.096123
[step 1610]: loss 1.075431
[step 1611]: loss 1.090688
[step 1612]: loss 1.153986
[step 1613]: loss 1.211445
[step 1614]: loss 1.176800
[step 1615]: loss 1.141769
[step 1616]: loss 1.141651
[step 1617]: loss 1.181733
[step 1618]: loss 1.141761
[step 1619]: loss 1.150178
[step 1620]: loss 1.121357
[step 1621]: loss 1.138588
[step 1622]: loss 1.125052
[step 1623]: loss 1.141512
[step 1624]: loss 1.168730
[step 1625]: loss 1.148114
[step 1626]: loss 1.171137
[step 1627]: loss 1.109164
[step 1628]: loss 1.164560
[step 1629]: loss 1.112042
[step 1630]: loss 1.173300
[step 1631]: loss 1.114514
[step 1632]: loss 1.119516
[step 1633]: loss 1.300208
[

...

[step 10980]: loss 0.998178
[step 10981]: loss 1.067893
[step 10982]: loss 1.026374
[step 10983]: loss 1.038963
[step 10984]: loss 0.990959
[step 10985]: loss 0.969498
[step 10986]: loss 0.935983
[step 10987]: loss 0.893998
[step 10988]: loss 0.953422
[step 10989]: loss 0.944133
[step 10990]: loss 0.983320
[step 10991]: loss 0.960091
[step 10992]: loss 1.045698
[step 10993]: loss 0.843536
[step 10994]: loss 0.923987
[step 10995]: loss 1.025030
[step 10996]: loss 0.946614
[step 10997]: loss 1.028515
[step 10998]: loss 0.999460
[step 10999]: loss 0.816677
[step 11000]: loss 1.000033
[step 11001]: loss 0.911399
[step 11002]: loss 0.917610
[step 11003]: loss 0.986918
[step 11004]: loss 0.911723
[step 11005]: loss 0.923863
[step 11006]: loss 0.943213
[step 11007]: loss 0.897540
[step 11008]: loss 1.028310
[step 11009]: loss 0.957843
[step 11010]: loss 0.916816
[step 11011]: loss 1.005757
[step 11012]: loss 0.973517
[step 11013]: loss 1.043341
[step 11014]: loss 0.876690
[step 11015]: loss 0

[step 11408]: loss 1.015336
[step 11409]: loss 1.033807
[step 11410]: loss 0.944311
[step 11411]: loss 0.911282
[step 11412]: loss 0.901901
[step 11413]: loss 1.000938
[step 11414]: loss 0.962319
[step 11415]: loss 0.947942
[step 11416]: loss 0.889856
[step 11417]: loss 1.009046
[step 11418]: loss 1.020128
[step 11419]: loss 0.945883
[step 11420]: loss 0.979783
[step 11421]: loss 0.914995
[step 11422]: loss 0.987920
[step 11423]: loss 0.867030
[step 11424]: loss 0.976914
[step 11425]: loss 0.896875
[step 11426]: loss 0.964637
[step 11427]: loss 0.978913
[step 11428]: loss 0.911486
[step 11429]: loss 0.931646
[step 11430]: loss 0.969265
[step 11431]: loss 0.959374
[step 11432]: loss 0.919420
[step 11433]: loss 0.889379
[step 11434]: loss 1.049776
[step 11435]: loss 1.010386
[step 11436]: loss 1.016940
[step 11437]: loss 1.000579
[step 11438]: loss 0.910890
[step 11439]: loss 0.969727
[step 11440]: loss 0.945353
[step 11441]: loss 0.909739
[step 11442]: loss 0.985107
[step 11443]: loss 0

[step 11756]: loss 0.781688
[step 11757]: loss 0.981237
[step 11758]: loss 1.030534
[step 11759]: loss 0.890011
[step 11760]: loss 0.859150
[step 11761]: loss 0.889950
[step 11762]: loss 0.934763
[step 11763]: loss 0.840528
[step 11764]: loss 0.951872
[step 11765]: loss 0.881677
[step 11766]: loss 0.965153
[step 11767]: loss 0.809717
[step 11768]: loss 0.971468
[step 11769]: loss 1.028131
[step 11770]: loss 0.959871
[step 11771]: loss 0.989625
[step 11772]: loss 0.977679
[step 11773]: loss 0.932918
[step 11774]: loss 0.886465
[step 11775]: loss 0.946264
[step 11776]: loss 0.893322
[step 11777]: loss 0.908007
[step 11778]: loss 0.988648
[step 11779]: loss 0.909495
[step 11780]: loss 1.000008
[step 11781]: loss 0.962804
[step 11782]: loss 0.939878
[step 11783]: loss 0.933930
[step 11784]: loss 0.955534
[step 11785]: loss 0.946701
[step 11786]: loss 0.893427
[step 11787]: loss 0.931070
[step 11788]: loss 0.839168
[step 11789]: loss 0.957020
[step 11790]: loss 0.891277
[step 11791]: loss 0

[step 12052]: loss 0.932398
[step 12053]: loss 0.932906
[step 12054]: loss 0.858639
[step 12055]: loss 1.026096
[step 12056]: loss 0.847221
[step 12057]: loss 0.943274
[step 12058]: loss 0.935051
[step 12059]: loss 0.918401
[step 12060]: loss 0.923841
[step 12061]: loss 0.847347
[step 12062]: loss 1.011090
[step 12063]: loss 0.871814
[step 12064]: loss 0.920535
[step 12065]: loss 0.946351
[step 12066]: loss 0.985893
[step 12067]: loss 1.012867
[step 12068]: loss 0.986278
[step 12069]: loss 0.956027
[step 12070]: loss 1.073880
[step 12071]: loss 1.046414
[step 12072]: loss 0.979170
[step 12073]: loss 1.016600
[step 12074]: loss 1.022271
[step 12075]: loss 0.924269
[step 12076]: loss 0.981817
[step 12077]: loss 0.956877
[step 12078]: loss 0.865584
[step 12079]: loss 0.972955
[step 12080]: loss 0.936157
[step 12081]: loss 0.907755
[step 12082]: loss 0.916588
[step 12083]: loss 0.875547
[step 12084]: loss 0.929017
[step 12085]: loss 1.026514
[step 12086]: loss 0.865460
[step 12087]: loss 1

[step 12404]: loss 0.789708
[step 12405]: loss 0.949286
[step 12406]: loss 1.020593
[step 12407]: loss 1.071849
[step 12408]: loss 0.868311
[step 12409]: loss 0.996457
[step 12410]: loss 0.887753
[step 12411]: loss 0.888743
[step 12412]: loss 1.008084
[step 12413]: loss 0.894257
[step 12414]: loss 0.838865
[step 12415]: loss 0.929304
[step 12416]: loss 0.960110
[step 12417]: loss 1.028043
[step 12418]: loss 0.911230
[step 12419]: loss 1.019438
[step 12420]: loss 0.937417
[step 12421]: loss 0.888924
[step 12422]: loss 1.012405
[step 12423]: loss 0.971008
[step 12424]: loss 0.947595
[step 12425]: loss 0.844692
[step 12426]: loss 1.011153
[step 12427]: loss 0.916322
[step 12428]: loss 0.925350
[step 12429]: loss 1.021916
[step 12430]: loss 1.003413
[step 12431]: loss 0.963244
[step 12432]: loss 0.978839
[step 12433]: loss 0.929839
[step 12434]: loss 1.007274
[step 12435]: loss 0.900115
[step 12436]: loss 0.950880
[step 12437]: loss 0.919314
[step 12438]: loss 0.886938
[step 12439]: loss 1

In [7]:
svd.test()

test loss:  1.1988026148573137
